In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 8, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((8, 8, 256)))
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(256, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(128, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(16, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(8, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(1,1), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator(num_classes):

        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        
        model.summary()
        
        img = Input(shape=img_shape)

        features = model(img)

        validity = Dense(1)(features)
        valid = Activation('sigmoid')(validity)
        
        label1 = Dense(512)(features)
        lrelu1 = LeakyReLU(alpha=0.2)(label1)
        label2 = Dense(256)(label1)
        lrelu2 = LeakyReLU(alpha=0.2)(label2)
        label3 = Dense(num_classes)(label2)
        label = Activation('softmax')(label3)
        
        return Model(img, valid), Model(img, label)

In [ ]:
def generator_containing_discriminator(g, d, d_label):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    d_label.trainable = False
    valid, target_label = d(img), d_label(img)
    
    
    return Model(noise, [valid, target_label])

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_label = np.zeros(batch_size)
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(4):
        for j in range(4):
            temp_img = cv2.imread(data[i][j])
            
            X_train[4*i + j] = temp_img
            y_label[4*i + j] = i
            
    X_train = (X_train - 127.5)/127.5
    return X_train, y_label

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4, weights = False):

    data, num_styles = get_data()
    
    num_styles = 4
    
    generator = build_generator()
    discriminator, d_label = build_discriminator(num_styles)
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    
    if weights:
        generator.load_weights(os.getcwd() + '24.11 gen_weights.h5')
        discriminator.load_weights(os.getcwd() + '24.11 dis_weights.h5')
        
    
    dcgan = generator_containing_discriminator(generator, discriminator, d_label)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_one_class(BATCH_SIZE, data, label)
            
            real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
                
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
                
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 1/num_styles, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, [y, y_classif])
            
            d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f, label_loss: %f" % (epoch, index, g_loss[0], g_loss[1]))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '/%d.%d dis_weights.h5' % (day, month))
            d_label.save_weights(os.getcwd() + '%d.%d dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '/%d.%d gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.0002, beta_2 = 0.5)

In [ ]:
train_another(1000, 16, weights = False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1654784   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 512)         3277312   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
__________

D:\programming\Anaconda1\envs\NN\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch 0 batch 0 d_loss : 3.933055
epoch 0 batch 0 g_loss : 2.379110, label_loss: 2.306799
epoch 0 batch 1 d_loss : 2.307128
epoch 0 batch 1 d_loss : 0.458502, label_loss: 3.426814
epoch 0 batch 1 g_loss : 1.718628, label_loss: 0.736007
epoch 0 batch 2 d_loss : 1.119753, label_loss: 1.947631
epoch 0 batch 2 d_loss : 0.578767
epoch 0 batch 2 g_loss : 3.054901, label_loss: 1.133246
epoch 0 batch 3 d_loss : 0.760334
epoch 0 batch 3 d_loss : 1.541170, label_loss: 2.381868
epoch 0 batch 3 g_loss : 3.002998, label_loss: 1.217175
epoch 0 batch 4 d_loss : 1.308724, label_loss: 1.181764
epoch 0 batch 4 d_loss : 0.783572
epoch 0 batch 4 g_loss : 2.265341, label_loss: 1.076524
epoch 0 batch 5 d_loss : 0.640697
epoch 0 batch 5 d_loss : 0.763387, label_loss: 1.571708
epoch 0 batch 5 g_loss : 2.392319, label_loss: 1.796821
epoch 0 batch 6 d_loss : 1.114947, label_loss: 1.912180
epoch 0 batch 6 d_loss : 5.492583
epoch 0 batch 6 g_loss : 4.966573, label_loss: 4.451204
epoch 0 batch 7 d_loss : 3.668474


epoch 0 batch 55 g_loss : 0.471376, label_loss: 0.434008
epoch 0 batch 56 d_loss : 0.361621, label_loss: 2.649297
epoch 0 batch 56 d_loss : 0.399650
epoch 0 batch 56 g_loss : 0.608436, label_loss: 0.575963
epoch 0 batch 57 d_loss : 0.461543
epoch 0 batch 57 d_loss : 0.326777, label_loss: 4.197580
epoch 0 batch 57 g_loss : 0.776234, label_loss: 0.596005
epoch 0 batch 58 d_loss : 0.348050, label_loss: 1.151891
epoch 0 batch 58 d_loss : 0.329468
epoch 0 batch 58 g_loss : 0.822771, label_loss: 0.422736
epoch 0 batch 59 d_loss : 0.310863
epoch 0 batch 59 d_loss : 0.306923, label_loss: 2.748506
epoch 0 batch 59 g_loss : 0.937449, label_loss: 0.426235
epoch 0 batch 60 d_loss : 0.267532, label_loss: 2.705431
epoch 0 batch 60 d_loss : 0.370367
epoch 0 batch 60 g_loss : 1.166365, label_loss: 0.438030
epoch 0 batch 61 d_loss : 0.420454
epoch 0 batch 61 d_loss : 0.317613, label_loss: 2.143092
epoch 0 batch 61 g_loss : 1.634540, label_loss: 0.508716
epoch 0 batch 62 d_loss : 0.397949, label_loss: 2

epoch 0 batch 110 g_loss : 0.543379, label_loss: 0.543379
epoch 0 batch 111 d_loss : 0.303816
epoch 0 batch 111 d_loss : 0.385829, label_loss: 0.827545
epoch 0 batch 111 g_loss : 0.466244, label_loss: 0.466243
epoch 0 batch 112 d_loss : 0.359558, label_loss: 0.777513
epoch 0 batch 112 d_loss : 0.357084
epoch 0 batch 112 g_loss : 0.421403, label_loss: 0.421402
epoch 0 batch 113 d_loss : 0.293969
epoch 0 batch 113 d_loss : 0.338543, label_loss: 1.171252
epoch 0 batch 113 g_loss : 0.442472, label_loss: 0.442472
epoch 0 batch 114 d_loss : 0.344282, label_loss: 1.928046
epoch 0 batch 114 d_loss : 0.350274
epoch 0 batch 114 g_loss : 0.500484, label_loss: 0.500483
epoch 0 batch 115 d_loss : 0.308008
epoch 0 batch 115 d_loss : 0.386905, label_loss: 0.828410
epoch 0 batch 115 g_loss : 0.726585, label_loss: 0.485498
epoch 0 batch 116 d_loss : 0.474527, label_loss: 1.112194
epoch 0 batch 116 d_loss : 0.350460
epoch 0 batch 116 g_loss : 0.556092, label_loss: 0.550821
epoch 0 batch 117 d_loss : 0.3

...

epoch 7 batch 155 g_loss : 0.423535, label_loss: 0.415737
epoch 7 batch 156 d_loss : 0.312757, label_loss: 0.460939
epoch 7 batch 156 d_loss : 0.332517
epoch 7 batch 156 g_loss : 0.414834, label_loss: 0.405927
epoch 7 batch 157 d_loss : 0.304992
epoch 7 batch 157 d_loss : 0.332712, label_loss: 0.610137
epoch 7 batch 157 g_loss : 0.460192, label_loss: 0.454690
epoch 7 batch 158 d_loss : 0.309581, label_loss: 1.573049
epoch 7 batch 158 d_loss : 0.363720
epoch 7 batch 158 g_loss : 0.457751, label_loss: 0.453153
epoch 7 batch 159 d_loss : 0.322624
epoch 7 batch 159 d_loss : 0.280967, label_loss: 1.542526
epoch 7 batch 159 g_loss : 0.471020, label_loss: 0.464534
epoch 7 batch 160 d_loss : 0.309204, label_loss: 1.560186
epoch 7 batch 160 d_loss : 0.307948
epoch 7 batch 160 g_loss : 0.384253, label_loss: 0.377368
epoch 7 batch 161 d_loss : 0.379221
epoch 7 batch 161 d_loss : 0.373600, label_loss: 0.541528
epoch 7 batch 161 g_loss : 0.469122, label_loss: 0.459789
epoch 7 batch 162 d_loss : 0.3

epoch 7 batch 209 g_loss : 0.437520, label_loss: 0.434349
epoch 7 batch 210 d_loss : 0.302303, label_loss: 0.577740
epoch 7 batch 210 d_loss : 0.341953
epoch 7 batch 210 g_loss : 0.507119, label_loss: 0.506982
epoch 7 batch 211 d_loss : 0.376615
epoch 7 batch 211 d_loss : 0.366341, label_loss: 1.579557
epoch 7 batch 211 g_loss : 0.411515, label_loss: 0.408454
epoch 7 batch 212 d_loss : 0.320424, label_loss: 0.555767
epoch 7 batch 212 d_loss : 0.311903
epoch 7 batch 212 g_loss : 0.487551, label_loss: 0.485864
epoch 7 batch 213 d_loss : 0.366891
epoch 7 batch 213 d_loss : 0.334803, label_loss: 0.483590
epoch 7 batch 213 g_loss : 0.468256, label_loss: 0.466303
epoch 7 batch 214 d_loss : 0.319009, label_loss: 0.490370
epoch 7 batch 214 d_loss : 0.306479
epoch 7 batch 214 g_loss : 0.525367, label_loss: 0.524840
epoch 7 batch 215 d_loss : 0.349458
epoch 7 batch 215 d_loss : 0.341682, label_loss: 1.498270
epoch 7 batch 215 g_loss : 0.344457, label_loss: 0.341055
epoch 7 batch 216 d_loss : 0.3

epoch 7 batch 263 g_loss : 0.374875, label_loss: 0.370635
epoch 7 batch 264 d_loss : 0.315700, label_loss: 0.541932
epoch 7 batch 264 d_loss : 0.330812
epoch 7 batch 264 g_loss : 0.414675, label_loss: 0.410973
epoch 7 batch 265 d_loss : 0.304454
epoch 7 batch 265 d_loss : 0.367853, label_loss: 0.578369
epoch 7 batch 265 g_loss : 0.469235, label_loss: 0.464664
epoch 7 batch 266 d_loss : 0.325026, label_loss: 1.606591
epoch 7 batch 266 d_loss : 0.364154
epoch 7 batch 266 g_loss : 0.579623, label_loss: 0.574258
epoch 7 batch 267 d_loss : 0.325264
epoch 7 batch 267 d_loss : 0.397350, label_loss: 1.618431
epoch 7 batch 267 g_loss : 0.468146, label_loss: 0.459733
epoch 7 batch 268 d_loss : 0.392906, label_loss: 0.544562
epoch 7 batch 268 d_loss : 0.331577
epoch 7 batch 268 g_loss : 0.443973, label_loss: 0.438110
epoch 7 batch 269 d_loss : 0.290690
epoch 7 batch 269 d_loss : 0.366920, label_loss: 0.425131
epoch 7 batch 269 g_loss : 0.495733, label_loss: 0.487297
epoch 7 batch 270 d_loss : 0.3

epoch 7 batch 317 g_loss : 0.399850, label_loss: 0.391210
epoch 7 batch 318 d_loss : 0.361757, label_loss: 0.687639
epoch 7 batch 318 d_loss : 0.303418
epoch 7 batch 318 g_loss : 0.460003, label_loss: 0.430124
epoch 7 batch 319 d_loss : 0.307525
epoch 7 batch 319 d_loss : 0.285318, label_loss: 0.477825
epoch 7 batch 319 g_loss : 0.469539, label_loss: 0.434016
epoch 7 batch 320 d_loss : 0.377085, label_loss: 1.173302
epoch 7 batch 320 d_loss : 0.325189
epoch 7 batch 320 g_loss : 0.542373, label_loss: 0.519170
epoch 7 batch 321 d_loss : 0.324290
epoch 7 batch 321 d_loss : 0.413367, label_loss: 0.510203
epoch 7 batch 321 g_loss : 0.400691, label_loss: 0.385717
epoch 7 batch 322 d_loss : 0.248894, label_loss: 1.204395
epoch 7 batch 322 d_loss : 0.330889
epoch 7 batch 322 g_loss : 0.446027, label_loss: 0.419355
epoch 7 batch 323 d_loss : 0.297969
epoch 7 batch 323 d_loss : 0.316103, label_loss: 0.538940
epoch 7 batch 323 g_loss : 0.478988, label_loss: 0.459465
epoch 7 batch 324 d_loss : 0.3

epoch 7 batch 371 g_loss : 1.717024, label_loss: 0.428029
epoch 7 batch 372 d_loss : 0.307881, label_loss: 0.597118
epoch 7 batch 372 d_loss : 0.336715
epoch 7 batch 372 g_loss : 1.721377, label_loss: 0.457843
epoch 7 batch 373 d_loss : 0.285513
epoch 7 batch 373 d_loss : 0.320345, label_loss: 1.490893
epoch 7 batch 373 g_loss : 1.496217, label_loss: 0.383763
epoch 7 batch 374 d_loss : 0.305859, label_loss: 1.511657
epoch 7 batch 374 d_loss : 0.314609
epoch 7 batch 374 g_loss : 1.381037, label_loss: 0.460346
epoch 7 batch 375 d_loss : 0.303001
epoch 7 batch 375 d_loss : 0.411391, label_loss: 0.480448
epoch 7 batch 375 g_loss : 1.311395, label_loss: 0.467360
epoch 7 batch 376 d_loss : 0.247179, label_loss: 1.440692
epoch 7 batch 376 d_loss : 0.355655
epoch 7 batch 376 g_loss : 1.148200, label_loss: 0.387245
epoch 7 batch 377 d_loss : 0.356218
epoch 7 batch 377 d_loss : 0.365703, label_loss: 0.605400
epoch 7 batch 377 g_loss : 1.129517, label_loss: 0.506750
epoch 7 batch 378 d_loss : 0.3

In [ ]:
current_moment = datetime.now()

day, month = str(current_moment.day), str(current_moment.month)
hour, minute = str(current_moment.hour), str(current_moment.minute)

with open(os.getcwd() + '/Changes.txt', 'a') as f:
    clock =  (day + '.' + month + '    ' + hour + ':' + minute)
    f.write(clock + '...........g_optim -> 0.0003, d_optim->0.0004' + '\n')